In [ ]:
pip install numpy

In [ ]:
pip install abjad

In [3]:
from ipywidgets import interact
from random import choice, randrange
from numpy.random import choice as npchoice
import abjad

In [4]:

pitches = range(12)
counts = [0] * 12 # counts since that index of pitches was chosen
pcprob = [1] * 12 # pitch class probability
weights = [1] * 12 # how much to weigh each element's probability

offset = int(input("Pitch Offset: "))
length = int(input("Number of pitches in sequence: ")) # number of notes, for now—can determine length in other ways
    


In [ ]:
def make_random_pitch_sequence(pitches, offset, length):
    sequence = []
    for note in range(length):
        note = choice(pitches)
        sequence.append(note + offset)
    return sequence

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [ ]:
print(make_random_pitch_sequence(pitches, offset, length))

# Notice the repetitions

In [ ]:
def get_probs(weights, counts):
    probs = []
    total = 0
    for i in range(len(weights)):
        total += weights[i] * growth_function(counts[i])

    for i in range(len(weights)):
        prob = weights[i] * growth_function(counts[i]) / total
        probs.append(prob)

    return probs

In [ ]:
def growth_function(n): # Tenney's convex growth function
    return 2**n

In [ ]:
def make_dca_spaced_pitch_sequence(pitches, offset, weights, length, show_data=False):
    counts = [0] * len(pitches)
    pcprobs = get_probs(weights, counts)

    sequence = []
    for note in range(length):
        # random choice according to probability
        note_choice = npchoice(pitches, p=pcprobs)
        element = pitches.index(note_choice)
        note = note_choice + offset

        # add note to sequence
        sequence.append(note)

        # now update counts
        for i, pitch in enumerate(pitches):
            if i == element:
                counts[i] = 0
            else:
                counts[i] += 1

        # recalculate pcprobs
        pcprobs = get_probs(weights, counts)

        if show_data:
            print("\nRandom index choice:", note_choice)
            print("Current counts:", counts)
            print("Current probabilities:", pcprobs)

    return sequence

In [ ]:
print(make_dca_spaced_pitch_sequence(pitches, offset, weights, length, show_data=True))

# After an element is chosen, it's count goes to 0 and it's probability goes to near-zero. Higher counts are biased by the growth function (2**n) to have even higher probabilities.

In [ ]:
length = int(input("Length: "))
sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
print(sequence)

# See any repetitions? Probably not!

# Here's what the pitch frequency looks like:

In [ ]:
for pitch in pitches:
    print(sequence.count(pitch))

# what if we change the weights to favor certain pitches?

In [ ]:
weights = [i * 2 for i in range(1, 13)]

In [ ]:
sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
print(sequence)

In [ ]:
for pitch in pitches:
    print(sequence.count(pitch))

# Still some randomness, but definitely skewed towards one side

# Let's reset the weights to an equal value and try that again

In [ ]:
weights = [1] * 12 # how much to weigh each element's probability

# Now let's see what this looks like in notation

In [ ]:
def output_ly(sequence):
    notes = []
    for pitch in sequence:
        duration = abjad.Duration(1, 4)
        note = abjad.Note(pitch, duration)
        notes.append(note)

    staff = abjad.Staff(notes)
    abjad.show(staff)

In [ ]:
sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
output_ly(sequence)

# How about changing over time?

1. make weights a function of their distance from a pitch center
2. move that pitch center over time
3. make sequences (clangs) vary in length
4. increase pitch center and sequence average length until 2/3 through form, then decrease

In [ ]:
def make_clang(pitches, weights, length, deviation, offset=0):
    # length = average length +/- deviation range
    length = length + randrange(deviation*-1, deviation)
    sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
    sequence.append("rest") # end every clang with a rest
    return sequence

In [ ]:
# build clangs
total_clangs = 72

# weight by proximity to pitch center
increasing = [ 1 * i for i in range(1, 7) ]
decreasing = list(reversed(increasing))
weights = increasing + decreasing

# make peak 2/3 of the way through form. Tenney does this with a calculation based on the golden mean, but is more/less in the same place
peak = int(total_clangs * (2/3))

length = 5
length_deviation = int(length * 0.5)

offset = -12
pitch_center = 6
clangs = []

In [ ]:
for clang in range(total_clangs):
    pitches = [*range(int(pitch_center) - 6, int(pitch_center+6))]
    #print(pitches)

    sequence = make_clang(pitches, weights, length, length_deviation, offset=offset)
    clangs.append(sequence)

    if clang < peak:
        pitch_center += 0.6
        length += 1
    else:
        pitch_center -= 1.25
        length -= 2
    length_deviation = int(length * 0.5)

In [ ]:
for clang in clangs:
    print(clang)

In [ ]:
def output_clangs(clangs):
    notes = []
    for clang in clangs:
        for pitch in clang:
            if pitch == "rest":
                notes.append(abjad.Rest('r8'))
            else:
                duration = abjad.Duration(1, 8)
                note = abjad.Note(pitch, duration)
                notes.append(note)

    staff = abjad.Staff(notes)
    abjad.show(staff)
    abjad.play(staff)

In [ ]:
output_clangs(clangs)